## Install Dependencies

In [1]:
!pip3 install --upgrade pip
# !pip3 install gcsfs==2024.6.1
!pip3 install transformers==4.48.1 datasets==3.2.0 tokenizers==0.21.0 accelerate==1.3.0 bitsandbytes==0.45.1 sentencepiece==0.2.0 torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 tqdm==4.66.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 158.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 178.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 31.3 MB/

## Login to Huggingface

In [2]:
from huggingface_hub import login
login(token="")

## Experiment 1 - Fine tuning a LLAMA 3 8b model on a customer support chat dataset.

### Load dataset

In [6]:
from datasets import load_dataset

In [7]:
dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset")

In [8]:
dataset['train']['instruction'][0:5]

['question about cancelling order {{Order Number}}',
 'i have a question about cancelling oorder {{Order Number}}',
 'i need help cancelling puchase {{Order Number}}',
 'I need to cancel purchase {{Order Number}}',
 'I cannot afford this order, cancel purchase {{Order Number}}']

### Model setup

In [10]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from torch import bfloat16

def load_llama_model(model_id):
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=bfloat16,
                                               device_map="auto")
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,
                  torch_dtype=bfloat16, device_map="auto", temperature=0.8)
  return pipe, model, tokenizer

In [11]:
pipe, model, tokenizer = load_llama_model("meta-llama/Llama-3.2-3B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0
